In [1]:
'''
The experiment computes the integral of the function sin(x) using various MC algorithms.
These MC algorithms can be used to compute probabilites from distributions where the 
PDF cannot be analytically integrated.
'''
import numpy as np

In [3]:
def required_sample(c, var, err):
    """
    Arguments:
    
    c   : confidence level
    var : variance of estimate
    err : tolerable error
    
    Returns the required sample size required 
    by an MC Algorithm to produce estimates
    with an error bound given a confidence level
    """
    return (np.power(c, 2)*var)/np.power(err, 2)


def cmc(n):
    """
    Arguments:
    
    n : number of samples
    
    Crude Monte Carlo estimation algorithm
    esitmates the value of an integral using
    basic Monte Carlo returns 
    """
    x = np.random.uniform(0, 1, n)
    g = np.sin(x)
    ans = np.mean(g)
    var = (np.var(g))/n
    return ans, var

In [4]:
def anti(n):
    """
    Arguments:
    
    n : number of samples
    
    Antithetic Monte-Carlo algorithm
    uses negative covariance variables
    to reduce variance of the estimate.
    """
    x = np.random.uniform(0, 1, n)
    y = 1 - x
    g = (np.sin(x) + np.sin(y))/2
    ans = np.mean(g)
    var = (np.var(g))/n
    return ans, var

In [5]:
def ocv(n):
    """
    Arguments:
    
    n : number of samples
    
    Optimal Control Variate algorithm
    uses a helper function h(x) to estimate
    the integral of g(x) and optimally
    adjusts for the error between the two.
    
    Useful if g(x) cannot be integrated
    analytically.
    """
    x = np.random.uniform(0, 1, n)
    g = np.sin(x)
    h = np.sqrt(x)
    M = np.stack((g, h), axis=0)
    k = (np.cov(M)[0][1])/np.var(h)
    ans = np.mean(g-k*h)+k*(2/3)
    var = np.var((g-k*h))/n
    return ans, var

In [6]:
def imp(n):
    """
    Arguments:
    
    n : number of samples
    
    Importance Sampling algorithm 
    uses a helper function h(x) and
    estimates the integral of h(x).
    Then uses this estimate to compute
    the integral of the g(x).
    
    Used to estimate tail probabilities
    with accuracy with a lower sample
    requirement than CMC.
    
    """
    u = np.random.uniform(0, 1, n)
    x = np.power(u, 2/3)
    h = (3/2)*np.sqrt(x)
    g = np.sin(x)
    ans = np.mean(g/h)
    var = (np.var(g/h))/n
    return ans, var

In [17]:
#Set Seed
np.random.seed(456)
#Compute Estimates and their Variances using different methods
est_cmc, var_cmc = cmc(1000)
est_imp, var_imp = imp(1000)
est_ocv, var_ocv = ocv(1000)

In [18]:
'''
Efficiency is given by the ratio of 
variances of the two methods.
For example below variance
of importance sampling is 
7.6 times the variance of
optimal control variates
'''
var_imp/var_ocv

7.594838913306361

In [19]:
#Required sample size for each of the methods
req_samp_cmc = required_sample(1.96, var_cmc, 0.0001)
req_samp_imp = required_sample(1.96, var_imp, 0.0001)
req_samp_ocv = required_sample(1.96, var_ocv, 0.0001)
print("Crude Monte-Carlo: %.2f" %req_samp_cmc)
print("Importance Sampling: %.2f" %req_samp_imp)
print("Optimal Control Variate: %.2f" %req_samp_ocv)

Crude Monte-Carlo: 24059.65
Importance Sampling: 3808.72
Optimal Control Variate: 501.49


In [63]:
print("Efficiency Analysis")
print("CMC v. IMP: %.2f" %(var_cmc/var_imp))
print("CMC v. OCV: %.2f" %(var_cmc/var_ocv))
print("IMP v. OCV: %.2f" %(var_imp/var_ocv))

Efficiency Analysis
CMC v. IMP: 6.32
CMC v. OCV: 47.98
IMP v. OCV: 7.59


In [25]:
'''
Lipschitz Analysis

The output of the robustness measure is sensitive
to k the Lipschitz constant. Hence the choice of k
is very important if k is large then the Lipschitz
condition will always be satisfied over a part of
the domain of f().

'''
def f(x):
    return x**2

def two_norm(x, y):
    return np.linalg.norm(x-y, 2)

In [43]:
x = np.random.uniform(1,10,(5,1))
y = np.random.uniform(1,10,(5,1))

In [44]:
x

array([[6.15508711],
       [5.77881935],
       [4.53548733],
       [9.41010637],
       [6.45858956]])

In [45]:
y

array([[5.24255308],
       [5.56570279],
       [2.8959335 ],
       [7.39914442],
       [1.20984247]])

In [47]:
np.sqrt(x**2-y**2)

array([[3.22501698],
       [1.55489727],
       [3.49058941],
       [5.81401443],
       [6.344262  ]])

In [57]:
g = f(x)**2-f(y)**2
g = np.sqrt(g)

In [62]:
h = 10*np.sqrt(x**2-y**2)
g<h

array([[ True],
       [ True],
       [ True],
       [False],
       [ True]])